# Gathering Data

In [1]:
from functions import *
from scivol import *
from helpers import *
import ipyvolume as ipv
import numpy as np
import json
import ants
import gzip

In [2]:
proj_root = parent_directory()
print(proj_root)
mri_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
template_folder = "/Users/joachimpfefferkorn/repos/neurovolume/templates"
output_folder = os.path.join(proj_root, "output/")

default_cmap = 'nipy_spectral'

/Users/joachimpfefferkorn/repos/neurovolume


In [3]:
raw_anat_img = ants.image_read(mri_input_filepath)

In [4]:
print(raw_anat_img)
explore_3D_array(raw_anat_img.numpy(), cmap=default_cmap)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

# Skull Stripping
Using ANTs

[this notebook](https://github.com/Angeluz-07/MRI-preprocessing-techniques/blob/main/notebooks/09_brain_extraction_with_template.ipynb) is what we're basing this on

## Load Template and Mask

In [5]:
mni_template_img = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_template_mask = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")

In [6]:
print(mni_template_img)
explore_3D_array(mni_template_img.numpy(), cmap=default_cmap)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (197, 233, 189)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, 134.0, -72.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



interactive(children=(IntSlider(value=98, description='SLICE', max=196), Output()), _dom_classes=('widget-inte…

In [7]:
print(mni_template_mask)
explore_3D_array(mni_template_mask.numpy(), cmap=default_cmap)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (197, 233, 189)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, 134.0, -72.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



interactive(children=(IntSlider(value=98, description='SLICE', max=196), Output()), _dom_classes=('widget-inte…

## Register Template and Masks to Raw image
Here we warp the MNI template

In [8]:
template_warp_to_raw_anat = ants.registration(
    fixed=raw_anat_img,
    moving=mni_template_img, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [0x175b90bc8,0x11947fd28,1] -m mattes[0x175b90bc8,0x11947fd28,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x175b90bc8,0x11947fd28,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmph__vey87,0x175c52348,0x175b91108] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
All_Command_lines_OK
Using single precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Center of mass alignment using fixed image: 0x175b90bc8 and moving image: 0x11947fd28 (type = Euler3DTransform)
  Reading mask(s).
    Registration stage 0
      No fixed mask
      No moving mask
    Registration stage 1
      No fixed mask
      No moving mask
  number of levels = 4
  number of levels = 3
  fixed image: 0x175b90bc8
  moving image: 0x11947fd28
  fixed image: 0x175b90bc8
  moving image: 0x11947fd28

In [9]:
registered_template_img = template_warp_to_raw_anat['warpedmovout']

The following comparison function asserts that the new `registered_template_img` image has been transformed to the size and shape of the `raw_anat_img`

In [10]:
explore_3D_array_comparison(
    arr_before=raw_anat_img.numpy(),
    arr_after=registered_template_img.numpy()
)
#I don't love "before" and "after" as the names here as we are really only changing the template
# This is pretty confusing, so just ignore the names for now
# Maybe I'll change this and write my own helper functions

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

Next, we apply this to the mask so that it fits over the brain in the data we wish to visualize

In [11]:
brain_mask = ants.apply_transforms(
    fixed=template_warp_to_raw_anat['warpedmovout'],
    moving=mni_template_mask,
    transformlist=template_warp_to_raw_anat['fwdtransforms'],
    interpolator='nearestNeighbor',
    verbose=True
)

['-d', '3', '-i', '0x175b91108', '-o', '0x175c525e8', '-r', '0x175c51aa8', '-n', 'nearestNeighbor', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmph__vey871Warp.nii.gz', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmph__vey870GenericAffine.mat']
Using double precision for computations.
Input scalar image: 0x175b91108
Could not create ImageIO for the input file, assuming dimension = 3 and scalar pixel type
Reference image: 0x175c51aa8
The composite transform comprises the following transforms (in order): 
  1. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmph__vey870GenericAffine.mat (type = AffineTransform)
  2. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmph__vey871Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpolateImageFunction
Output warped image: 0x175c525e8


In [12]:
explore_3D_array_with_mask_contour(raw_anat_img.numpy(), brain_mask.numpy())

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

Let's fine tune the mask a bit

In [13]:
brain_mask_dilated = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')

In [14]:
explore_3D_array_with_mask_contour(raw_anat_img.numpy(), brain_mask_dilated.numpy())

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

# Mask the Image and Create Sections

In [15]:
brain_anat = ants.mask_image(raw_anat_img, brain_mask_dilated)

In [16]:
print(brain_anat)
explore_3D_array(brain_anat.numpy(), cmap=default_cmap)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

# Building Scivol

It looks like ANTS already applies the affine (double check this), so I think you can just 86 that from your workflow

Initialize the scivol object

In [17]:
skullstrip = Scivol("skullstrip")

Create the volumes:

In [18]:
full_anat_vol = create_normalized_volume(raw_anat_img.numpy())
brain_anat_vol = create_normalized_volume(brain_anat.numpy())

Create the grids:

In [19]:
explore_3D_array(full_anat_vol)
explore_3D_array(brain_anat_vol)

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

In [20]:
skullstrip.add_grids([Grid("full_anat",[full_anat_vol]), Grid("brain_anat", [brain_anat_vol])])

writing full_anat to grid
writing brain_anat to grid


In [21]:
print(skullstrip)
skullstrip.save_scivol(output_folder)

skullstrip
tolerance: 0.0
affine: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
grids:
    - full_anat
    - brain_anat

writing skullstrip scivol dictionary
Encoding JSON file to utf-8
saving skullstrip.scivol to /Users/joachimpfefferkorn/repos/neurovolume/output/


Lets' quickly test this in our notebook

In [24]:
test_compressed_file_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/skullstrip.scivol"
with gzip.open(test_compressed_file_path, 'r') as svin:
    scivol_bytes = svin.read()
scivol_str = scivol_bytes.decode('utf-8')

In [25]:
scivol_dict = json.loads(scivol_str)

In [26]:
print(type(scivol_dict))
print(scivol_dict['name'])
print([grid for grid in scivol_dict['grids']])
brain_anat_test_vol = np.asanyarray(scivol_dict['grids']['brain_anat']['frames'][0])
print(type(brain_anat_test_vol))
explore_3D_array(brain_anat_test_vol)

<class 'dict'>
skullstrip
['full_anat', 'brain_anat']
<class 'numpy.ndarray'>


interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

Before moving to blender to render these images, let's make sure that we can parse these scivol files here